# Pregunta 55

La cantidad promedio de modificaciones históricas que tuvieron los ítems cuya última versión fue editada por un usuario registrado o no registrado.

## Supuestos

* Se interpreta por 'item' como publicacion de contenido.
* `contents.csv` contiene la ultima version editada del contenido.
* `contents.csv` no tiene publicaciones repetidas, porque de haber habrian 2 ultimas modificaciones y no tiene sentido.
* `log.csv` contiene el historial de modificaciones hechas para cada publicacion de contenido.

In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

# 1. Creo un identificador unico para cada uno de los usuarios

Creo un identificador unico para cada usuario registrado o no registrado que hizo la ultima modificacion.

In [2]:
wiki_contents=pd.read_csv('./Wikipedia_dataset_2c2021/contents.csv', \
                          usecols=['revisor_id','revisor_ip', 'title'])

In [3]:
wiki_contents.revisor_id.fillna(wiki_contents.revisor_ip,inplace=True)
del wiki_contents["revisor_ip"]

#wiki_contents.drop(columns='revisor_ip')

In [4]:
wiki_contents = wiki_contents.rename(columns={"revisor_id":"identificador_editor"})

display(wiki_contents)

,title,identificador_editor
0,Wikipedia:Artículos solicitados,"1,708,233.00"
1,Andorra,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2
2,Argentina,"4,642,283.00"
3,Geografía de Andorra,"5,338,508.00"
4,Demografía de Andorra,"3,615,366.00"
...,...,...
4132159,Cruz nestoriana,"474,907.00"
4132160,Manifestación del orgullo gay de 1977 en Barce...,"4,032,712.00"
4132161,Rocasaurus,"158,577.00"
4132162,Medalla Conmemorativa del 50.º Aniversario de ...,"5,429,951.00"


# 2. Evaluo si hay datos invalidos y los borro en caso de haber

En este paso, me quedo con las publicaciones (items) cuya ultima version fue editada por un usuario registrado o no registrado.

In [5]:
wiki_contents["identificador_editor"].hasnans

True

In [6]:
wiki_contents.dropna(subset=['identificador_editor'], inplace=True)

In [7]:
wiki_contents["identificador_editor"].hasnans

False

In [8]:
wiki_contents["title"].hasnans

True

In [9]:
wiki_contents.dropna(subset=['title'], inplace=True)

In [10]:
wiki_contents["title"].hasnans

False

# 3. Intersecto los contenidos modificados con los logs

Quiero el historial de las publicaciones de contenido que fueron editadas. 

Para eso intersecto por `title` al database de `log.csv` con la database `wiki_contents` recien hallado

In [ ]:
# Cargo los datasets
logs = pd.read_csv('./Wikipedia_dataset_2c2021/logs.csv', usecols=['title', 'action'])
display(logs)

In [ ]:
# Determino cuantas veces se modifica cada publicacion
logs_db = logs.groupby('title').agg(cant_modificaciones=('title', 'count') )
display(logs_db)

### Uno los databases

In [ ]:
contenido_con_cant_modificaciones = wiki_contents.merge(logs_db, how='inner', left_on='title',right_index=True)
display(contenido_con_cant_modificaciones)

# 4. Cantidad promedio de modificaciones

In [ ]:
contenido_con_cant_modificaciones['cant_modificaciones'].mean()

## Estadisticas Adicionales

In [ ]:
contenido_con_cant_modificaciones.groupby('cant_modificaciones').agg({'title':'count'}) / \
                                    contenido_con_cant_modificaciones['cant_modificaciones'].size * 100

In [ ]:
# Porcentaje de publicaciones que se modifican 5 o mas veces

contenido_con_cant_modificaciones.groupby('cant_modificaciones').agg({'title':'count'}).loc[5:,].sum() / \
                                    contenido_con_cant_modificaciones['cant_modificaciones'].size * 100

# Conclusion

Vemos que el promedio de cantidad de modificaciones historicas para las publicaciones cuya ultima version fue editada por cualquier tipo de usuario (registrado o no registrado) es de 1.43.

Esto significa que un porcentaje alto de publicaciones no se editan mucho una vez creada.

Para la muestra de publicaciones estudiada (las ultimas modificadas por usuarios), vemos que un 77.21% son publicaciones que solo se modifican 1 vez (seguramente en su creacion) y la inmensa mayoria se editan como maximo 5 veces.

Es interesante ver que aproximadamente un 2.17% del total de 1193006 publicaciones tienen modificaciones mayor o igual a 5 veces.

Resalta el outlier de la publicacion que ha sido modificada 521 veces. Esta publicacion corresponde a la portada de Wikipedia, lo cual es razonable que sea modificada muchas veces.